# DEPENDENCIAS Y LIBRERIAS

In [ ]:
!pip install pypdf langchain sentence-transformers openai pinecone -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.6 MB/s eta 0:00:00


# PINECONE


Es base de datos vectorial en la nube diseñada para trabajar con inteligencia artificial y machine learning. Se utiliza principalmente para almacenar, buscar y recuperar vectores o representaciones numéricas de datos como textos, imágenes o audio de manera rápida y eficiente.

In [ ]:
# IMPORTAR CREDENCIALES DE PINECONE
from pinecone import Pinecone
import os

os.environ['PINECONE_API_KEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
pinecone = Pinecone(api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
INDEX_NAME = 'proyectonlp'

In [ ]:
# Estado de la base de datos
pinecone.list_indexes()

[
    {
        "name": "proyectonlp",
        "metric": "cosine",
        "host": "proyectonlp-1e27haj.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [ ]:
# Descripcion de los indices
index_description = pinecone.describe_index(INDEX_NAME)

In [ ]:
# Nombre del indice
index_description.name

'proyectonlp'

In [ ]:
# Dimension de los vectores
index_description.dimension

384

In [ ]:
# Metrica de los vectores
index_description.metric

'cosine'

Cargar vectores

In [ ]:
#  Librerias
import numpy as np

In [ ]:
# Crear dos vectores aleatorios para comprobar conexion
vector_1 = np.random.uniform(-1, 1,384).tolist()
vector_2 = np.random.uniform(-1, 1,384).tolist()

In [ ]:
# Establecer conexion
index = pinecone.Index(INDEX_NAME)

In [ ]:
# Cargar vectores creados
upsert_response = index.upsert(
   vectors=[
       {'id': "vec1", "values":vector_1, "metadata": {'genre': 'cine'}},
       {'id': "vec2", "values":vector_2, "metadata": {'genre': 'teatro'}},
   ]
)

Buscar vectores

In [ ]:
# Genero un vector pregunta aleatorio
vector_pregunta = np.random.uniform(-1, 1, 384).astype(np.float32).tolist()

In [ ]:
# Obtengo cual es el vector mas parecido
result = index.query(
    vector=vector_pregunta,
    top_k=1,
    include_metadata=True
)

print(result)

{'matches': [{'id': '1add3d46-0d74-4da9-96ce-ec136a0fe18f',
              'metadata': {'author': 'MAFFEIS_A',
                           'creationdate': '2015-01-20T13:57:44+01:00',
                           'creator': 'PScript5.dll Version 5.2.2',
                           'moddate': '2015-01-20T14:03:49+01:00',
                           'page': 1.0,
                           'page_label': '2',
                           'producer': 'Acrobat Distiller 9.5.5 (Windows)',
                           'source': '/content/DCRL8_MANUAL.pdf',
                           'text': 'Output status \n'
                                   'Cooling fan \n'
                                   'status \n'
                                   'Secondary \n'
                                   'display \n'
                                   'Active alarm \n'
                                   'Automatic \n'
                                   'mode \n'
                                   'Manual \n'
         

Borrar vectores

In [ ]:
# Borrar un vector sabiendo su genero
index.delete(
    filter={
        "genre": "cine"
    }
)

{}

In [ ]:
# Borrar todos los vectores
index.delete(delete_all=True)

{}

# LEER EL PDF

In [ ]:
# DEPENDENCIAS
!pip install -U langchain langchain-community langchain-openai -q

# LIBRERIAS
from langchain.document_loaders import PyPDFLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Cargar fichero PDF
#FILE = "la-rosaleda_v3.pdf"
FILE = "/content/DCRL8_MANUAL.pdf"

In [ ]:
# trabajaremos con documents y no text
loader = PyPDFLoader(FILE)
doc = loader.load()

# CREAR CHUNKS

In [ ]:
# LIBRERIAS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# SEPARAR EN CHUNKS O BLOQUES DE TEXTO MAS PEQUENOS
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
    )

In [ ]:
# split_documents y no text
chunks = text_splitter.split_documents(doc)

In [ ]:
# LONGITUD DE CHUNKS
len(chunks)

137

In [ ]:
# FUNCION QUE CREA CHUNKS
def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )
    return text_splitter.split_documents(doc_to_chunk)

# EMBEDDINGS

In [ ]:
# LIBRERIAS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# CARGAR MODELOS
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11G
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipython-input-3242922350.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acc

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# CREAR EMBEDDING DE PRUEBA
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#sentence_embeddings = model.encode("¿Cuántas copas ha ganado el Málaga CF?")
sentence_embeddings = model.encode("¿Que funciones tiene el DCRL8?")


In [ ]:
# VER VECTOR CREADO
sentence_embeddings

array([-6.99380219e-01,  7.26888180e-02, -2.53188729e-01, -1.18411928e-01,
       -4.38079864e-01, -4.01247054e-01, -6.92596436e-02,  5.58007300e-01,
        2.32511684e-02, -1.75160974e-01,  3.49942744e-01,  1.44911055e-02,
        9.38090757e-02, -2.98435360e-01, -1.32024229e-01, -3.74077618e-01,
        5.02171293e-02,  2.69151121e-01, -1.33800328e-01, -2.66590178e-01,
        6.51828572e-02, -5.44808991e-02,  4.98811267e-02,  7.03664720e-02,
       -4.04504418e-01,  1.33792385e-01,  2.59611726e-01, -2.19659135e-02,
        9.30433571e-02, -2.30433509e-01, -1.57090932e-01,  1.17898747e-01,
       -5.57815805e-02,  1.88309252e-01,  1.33402362e-01, -2.07259115e-02,
       -1.74296737e-01, -2.14971781e-01,  1.48787424e-01, -3.66715789e-01,
       -1.90188944e-01,  1.15146674e-02,  4.36109334e-01,  4.12933528e-01,
        3.72923352e-02, -6.59958348e-02, -1.30420670e-01, -1.98470846e-01,
       -1.26789451e-01, -1.63441703e-01,  9.88476723e-02,  1.30785257e-01,
       -4.52044047e-02, -

In [ ]:
# LONGITUD DE EMBEDDING
len(sentence_embeddings)

384

In [ ]:
# CARGAR VECTOR A BASE DE DATOS
!pip install -U langchain-pinecone -q

from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,          # tus chunks tipo Document[]
    embedding=embeddings,      # instancia de embeddings (por ejemplo OpenAIEmbeddings)
    index_name=INDEX_NAME      # nombre de tu índice Pinecone
)

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


# PREGUNTAR AL GENERADOR DE TEXTO

In [ ]:
# CONECTAR A API DE CHATGPT
import os
os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
# LIBRERIAS
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
# VECTORES Y SELECCION DE MODELO
vstore = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
)

In [ ]:
# PREGUNTA CON BASE EN EL PDF
pregunta = "¿Cuántas copas ha ganado el Málaga?"
#pregunta = "¿Dónde está ubicado el monumento a la revolución?"
#pregunta = "¿Cuántos habitantes hay en la ciudad de México?"
pregunta = "¿Que funciones tiene el DCRL8?"

In [ ]:
# RESPUESTA DEL GPT
res = qa.invoke({"query": pregunta})
print(res["result"])

El DCRL8 es una unidad de control automático del factor de potencia que ofrece funciones avanzadas para aplicaciones de compensación del factor de potencia. Algunas de sus funciones incluyen la posibilidad de copiar ajustes de configuración entre el DCRL8 y la llave de programación CX02, clonar datos entre el DCRL8 y la CX02, mostrar información sobre los datos almacenados en la CX02, y salir del menú de la llave de programación.


In [ ]:
# PREGUNTA Y RESPUESTA TEST
pregunta = "¿Cuántos habitantes hay en la ciudad de México?"
res = qa.invoke({"query": pregunta})
print(res["result"])

Lo siento, no tengo esa información.


Anadir nuevos vectrores

In [ ]:
# ANADIR UN NUEVO VECTOR (A PARTIR DE TEXTO SE TRANSFORMA EN EMBEDDING Y SE CARGA A LA BASE COMO INFORMACION)
res = vstore.from_texts(
    ["En la Ciudad de México viven 20 millones de personas"],
    embeddings,
    metadatas=[{'doc':'deporte'}],
    index_name=INDEX_NAME)

In [ ]:
# PREGUNTA Y RESPUESTA BASADA EN EL NUEVO VECTOR
pregunta = "¿Cuántos habitantes hay en la ciudad de México?"
res = qa.invoke({"query": pregunta})
print(res["result"])

En la Ciudad de México viven 20 millones de personas.
